### calculate csw exp costs

In [23]:
import os, re
import pandas as pd
import numpy as np
import json

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 22

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

def get_sql_df(exp_version=None,verb=False):
  import sqlite3 as sql
  from glob import glob as glob
  db_fpath = 'csw_mturk_spring19.db'
  with sql.connect(db_fpath) as conn:
    # connection objecs represent the database
    # cursor objects point to rows in the database
    c = conn.cursor()

    # list tables in database
    c.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables_in_db = c.fetchall()
    table_name = "CSWfall18" # table name must be constant to prevent repeat subjects

    # table header
    c.execute("PRAGMA table_info(%s)"%table_name)
    col_names = [i[1] for i in c.fetchall()]

    # select everything within table
    db_datastring = c.execute("""SELECT * FROM %s"""%table_name).fetchall()
    sql_df = pd.DataFrame(db_datastring,columns=col_names)
    
    # remove unneeded rows 
    sql_df = sql_df[sql_df['mode'] != 'debug'] # debug rows
    sql_df = sql_df[pd.notnull(sql_df['datastring'])] # rows with no datastring 
    # return only select code version
    versions = sql_df.codeversion.unique()
    if verb:
      print('exp versions found in table:')
      for v in versions: print(v)
    # if version not specified, take latest
    if exp_version==None:
      exp_version = versions[-1]
    if verb:
      print('\n-loading code version:',exp_version)
    sql_df = sql_df[sql_df.codeversion == exp_version]
    if verb:
      print('N =',len(sql_df),'subjects')
  return sql_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
_ = get_sql_df(verb=True)

exp versions found in table:
10.13.18
1000cl
1000clq
9010cl
i1000cl
20B1000cl
RT01B1000cl
csw1000block40.04.07.19
csw1000block01.04.25.19
csw1000block02.04.26.19
csw1000rand50.04.29.19
csw1000block01intheloc.05.01.19
csw1000rand01.05.03.19
csw1000block02.05.07.19
csw1000boldmdp.05.09.19
csw1000insertedblock0.09.22.19
csw1000insertedblock1.09.30.19
csw1000insertedblock1.10.07.19
csw1000insertedblock0.10.08.19
csw1000insertedblock2.10.09.19
csw1000insertedblock2.10.10.19

-loading code version: csw1000insertedblock2.10.10.19
N = 56 subjects


In [33]:
dnameL = [
  'csw1000block40.04.07.19',
  'csw1000block01.04.25.19',
  'csw1000block02.04.26.19',
  'csw1000rand50.04.29.19',
  'csw1000block01intheloc.05.01.19',
  'csw1000rand01.05.03.19',
  'csw1000block02.05.07.19',
  'csw1000boldmdp.05.09.19',
  'csw1000insertedblock0.09.22.19',
  'csw1000insertedblock1.09.30.19',
  'csw1000insertedblock1.10.07.19',
  'csw1000insertedblock0.10.08.19',
  'csw1000insertedblock2.10.09.19',
  'csw1000insertedblock2.10.10.19',
]


In [35]:
for dname in dnameL:
  sql_df = get_sql_df(dname)
  total_exp_bonus = sql_df['bonus'].sum()
  nsubs,_ = sql_df.shape
  total_exp_cost = nsubs*4 + total_exp_bonus
  print(dname,nsubs,total_exp_cost)

csw1000block40.04.07.19 63 427.71000000000004
csw1000block01.04.25.19 56 382.29
csw1000block02.04.26.19 55 358.9
csw1000rand50.04.29.19 78 494.66999999999996
csw1000block01intheloc.05.01.19 52 363.63
csw1000rand01.05.03.19 50 351.81
csw1000block02.05.07.19 52 369.64
csw1000boldmdp.05.09.19 52 371.53
csw1000insertedblock0.09.22.19 51 312.81
csw1000insertedblock1.09.30.19 56 374.22
csw1000insertedblock1.10.07.19 51 351.75
csw1000insertedblock0.10.08.19 57 382.25
csw1000insertedblock2.10.09.19 51 351.58
csw1000insertedblock2.10.10.19 56 359.1
